In [ ]:
#install all the necessary dependencies

In [2]:
pip install tensorflow torch transformers langchain openai spacy nltk streamlit fastapi uvicorn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#here first set path for the dataset,that i took from kaggle Deepfake Image detection set
#Then, I used ImageData Generqator to preprocess and augment the Images

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/pavan/deepfake-dataset/train"
val_dir = "/home/pavan/deepfake-dataset/val"
test_dir = "/home/pavan/deepfake-dataset/test"

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 349 images belonging to 2 classes.
Found 74 images belonging to 2 classes.
Found 76 images belonging to 2 classes.


In [ ]:
#Built a CNN model(Convolutional Neural Network) a Deep Learning model.
#Later i Saved the model to our directory with name deepfake_model.h5
#To avoid the warnings you can save the model as deepfake_model.keras
#Then Train the model with training dataset

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.save('/home/pavan/cyfuture/deepfake_model.h5')

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)


/home/pavan/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 6/10 ━━━━━━━━━━━━━━━━━━━━ 3s 852ms/step - accuracy: 0.6385 - loss: 1.0753

/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.6812 - loss: 0.9521 - val_accuracy: 0.7812 - val_loss: 0.5056
Epoch 2/10
 1/10 ━━━━━━━━━━━━━━━━━━━━ 5s 580ms/step - accuracy: 0.8438 - loss: 0.4022

/home/pavan/.local/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step - accuracy: 0.8438 - loss: 0.4022 - val_accuracy: 0.7500 - val_loss: 0.7066
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 962ms/step - accuracy: 0.7513 - loss: 0.5920 - val_accuracy: 0.7812 - val_loss: 0.5018
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - accuracy: 0.7931 - loss: 0.4741 - val_accuracy: 0.7812 - val_loss: 0.4822
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 954ms/step - accuracy: 0.8070 - loss: 0.5032 - val_accuracy: 0.7812 - val_loss: 0.5650
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.7500 - loss: 0.5020 - val_accuracy: 0.7969 - val_loss: 0.5665
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 953ms/step - accuracy: 0.7701 - loss: 0.4404 - val_accuracy: 0.7969 - val_loss: 0.4437
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.8750 - loss: 0.3757 - val_accuracy: 0.7656 - val_loss: 0.4608
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 930ms/step - accuracy: 0.7587 - loss: 0.4294 - val_accuracy: 0.8125 - v

In [ ]:
# Then test the model with the test dataset and got the accuracy as 0.78

In [4]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 405ms/step - accuracy: 0.7827 - loss: 0.5593
Test Accuracy: 0.78


In [ ]:
#save the model after testing

In [5]:
model.save('/home/pavan/cyfuture/deepfake_model.h5')


In [ ]:
#using hugging face's transformers library  to generate text using gpt-2 model

In [21]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')

# to  Generate text
generated_text = generator("Deepfake detection is important because", max_length=50)
print(generated_text)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Deepfake detection is important because there is always a possibility that an individual could possibly be doing something maliciously," says Andrew B. Reimer, a professor at Northeastern University who specializes in software analysis and vulnerability research, in his book, "Sc'}]


In [20]:
pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:01:010m
Note: you may need to restart the kernel to use updated packages.


In [11]:
print(1)

1
